In [ ]:
# default_exp huqianghuiNN


#export
def showNNResult():
    import numpy as np
    import matplotlib. pyplot as plt

    plt.rcParams["figure.figsize"]=(10.0, 8.0)
    plt.rcParams["image.interpolation"] = "nearest"
    plt.rcParams["image.cmap"] = "gray"

    np.random.seed(0)
    N=100 # 样本数
    D=2 # 样本维数
    K=3 # 种类数

    # 样本对应种类初始化矩阵（300 * 2）
    X = np.zeros((N*K, D))

    # 样本分类初始化矩阵（300 * 1）
    y=np.zeros(N*K,dtype=np.uint8)

    # 样本随机值处理
    for j in range(K):
      ix = range(N*j,N*(j+1))
      r = np.linspace(0.0,1,N)  
      # t = np.linspace(j*4,(j+1)*4, N) + np.random.randn(N) * 0.2
      t = np.linspace(j * 4, (j + 1) * 4, N) + np.random.randn(N) * .2
      X[ix] = np.c_[r * np.sin(t), r * np.cos(t)]
      y[ix] = j

    h = 100 # 神经元个数
    W = 0.01 * np.random.randn(D,h)
    b = np.zeros((1,h))


    W2 = 0.01 * np.random.randn(h,K)
    b2 = np.zeros((1,K))


    import math

    #step_size = math.exp(-1)
    #reg=math.exp(-3)

    num_examples = X.shape[0]

    for i in range(2999):
      # 正向LuRe化 
      hidden_layer = np.maximum(0,np.dot(X,W) + b)
      # 计算得分  
      scores = np.dot(hidden_layer, W2) + b2
      exp_scores = np.exp(scores)
      probs = exp_scores/np.sum(exp_scores,axis=1, keepdims=True)

      corect_logprobs = -np.log(probs[range(num_examples), y])
      data_loss = np.sum(corect_logprobs)/num_examples
      #reg_loss = 0.5 * reg * np.sum(W*W) + 0.5 * reg * np.sum(W2 * W2)
      #loss = data_loss + reg_loss
      if i % 100 == 0:
        print("iteration {} loss {}".format(i,data_loss))

      #dscores = probs
      #dscores[range(num_examples),y] -= 1
      #dscores /= num_examples
      probs[range(num_examples), y] -= 1
      # 把整体切分成N等分
      dscores = probs / num_examples 

      #dW2 = np.dot(hidden_layer.T,dscores)
      #db2 = np.sum(dscores,axis=0,keepdims=True)

      dhidden = np.dot(dscores,W2.T)
      dhidden[hidden_layer <= 0] = 0


      # 6、参数更新：W2、b2 (参数梯度下降)
      W2 -= np.dot(hidden_layer.T, dscores)
      b2 -= np.sum(dscores, axis=0, keepdims=True)

      # 7、参数更新：W1、b1
      W -= np.dot(X.T, dhidden)
      b -= np.sum(dhidden, axis=0, keepdims=True)

      #dW2 += reg * W2
      #dW += reg * W

      #W += -step_size * dW
      #b += -step_size * db
      #W2 += -step_size * dW2
      #b2 += -step_size * db2


    hidden_layer = np.maximum(0,np.dot(X,W) + b)
    scores = np.dot(hidden_layer,W2) + b2
    predicted_class = np.argmax(scores,axis=1)
    print("training accuracy: {}".format(np.mean(predicted_class == y)))


    h=0.02
    x_min,x_max = X[:,0].min()-1,X[:,0].max() +1
    y_min,y_max = X[:,1].min()-1,X[:,1].max() +1
    xx,yy = np.meshgrid(np.arange(x_min,x_max,h),np.arange(y_min,y_max,h))

    Z = np.dot(np.maximum(0,np.dot(np.c_[xx.ravel(),yy.ravel()],W) + b),W2) + b2
    Z = np.argmax(Z,axis=1)
    Z = Z.reshape(xx.shape)
    fig = plt.figure()

    plt.contourf(xx,yy,Z,cmap=plt.cm.Spectral,alpha=0.8)
    plt.scatter(X[:,0],X[:,1],c=y,s=40,cmap=plt.cm.Spectral)
    plt.xlim(xx.min(),xx.max())
    plt.ylim(yy.min(),yy.max())
    # https://cs.stanford.edu/~karpathy/convnetjs/demo/classify2d.html
    plt.show()